In [1]:
import pandas as pd
import os

In [2]:
df = pd.read_csv('data/Building_Energy_Benchmarking.csv')

In [3]:
df.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [5]:
df = df[df['BuildingType'].isin(['NonResidential', 'Nonresidential COS','SPS-District K-12','Nonresidential WA','Campus'])]

In [6]:
df.shape

(1668, 46)

In [7]:
df = df[~df['PrimaryPropertyType'].isin(['Senior Care Community','Low-Rise Multifamily'])]

In [8]:
df.shape

(1645, 46)

In [9]:
df = df[df['Outlier'].isnull()]
df = df.drop(columns='Outlier')

In [10]:
df.shape

(1629, 45)

In [11]:
df['ComplianceStatus'].value_counts()

ComplianceStatus
Compliant                       1526
Error - Correct Default Data      86
Missing Data                      14
Non-Compliant                      3
Name: count, dtype: int64

In [12]:
df = df[df['ComplianceStatus']=='Compliant']
df = df.drop(columns='ComplianceStatus')

In [13]:
df.shape

(1526, 44)

In [14]:
df['SecondLargestPropertyUseType'] = df['SecondLargestPropertyUseType'].fillna("none")
df['SecondLargestPropertyUseTypeGFA'] = df['SecondLargestPropertyUseTypeGFA'].fillna(0)
df['ThirdLargestPropertyUseType'] = df['ThirdLargestPropertyUseType'].fillna("none")
df['ThirdLargestPropertyUseTypeGFA'] = df['ThirdLargestPropertyUseTypeGFA'].fillna(0)

In [15]:
df.duplicated().sum()

np.int64(0)

In [16]:
df.isna().sum()

OSEBuildingID                         0
DataYear                              0
BuildingType                          0
PrimaryPropertyType                   0
PropertyName                          0
Address                               0
City                                  0
State                                 0
ZipCode                              13
TaxParcelIdentificationNumber         0
CouncilDistrictCode                   0
Neighborhood                          0
Latitude                              0
Longitude                             0
YearBuilt                             0
NumberofBuildings                     0
NumberofFloors                        0
PropertyGFATotal                      0
PropertyGFAParking                    0
PropertyGFABuilding(s)                0
ListOfAllPropertyUseTypes             0
LargestPropertyUseType                4
LargestPropertyUseTypeGFA             4
SecondLargestPropertyUseType          0
SecondLargestPropertyUseTypeGFA       0


In [17]:
df.select_dtypes(include='number').lt(0).sum()

OSEBuildingID                         0
DataYear                              0
ZipCode                               0
CouncilDistrictCode                   0
Latitude                              0
Longitude                          1526
YearBuilt                             0
NumberofBuildings                     0
NumberofFloors                        0
PropertyGFATotal                      0
PropertyGFAParking                    0
PropertyGFABuilding(s)                0
LargestPropertyUseTypeGFA             0
SecondLargestPropertyUseTypeGFA       0
ThirdLargestPropertyUseTypeGFA        0
ENERGYSTARScore                       0
SiteEUI(kBtu/sf)                      0
SiteEUIWN(kBtu/sf)                    0
SourceEUI(kBtu/sf)                    0
SourceEUIWN(kBtu/sf)                  1
SiteEnergyUse(kBtu)                   0
SiteEnergyUseWN(kBtu)                 0
SteamUse(kBtu)                        0
Electricity(kWh)                      1
Electricity(kBtu)                     1


In [18]:
df = df[df['SiteEnergyUseWN(kBtu)']>0]
df = df[df['Electricity(kBtu)']>0]

In [19]:
col_to_drop = ['OSEBuildingID',
               'DataYear',
               'PropertyName',
               'Address',
               'City',
               'State',
               'ZipCode',
               'TaxParcelIdentificationNumber',
               'CouncilDistrictCode',
               'Latitude',
               'Longitude',
               'ListOfAllPropertyUseTypes',
               'DefaultData',
               'YearsENERGYSTARCertified',
               'Comments']

In [20]:
df = df.drop(columns=col_to_drop)

In [21]:
df.shape

(1513, 29)

In [22]:
df.to_csv('data/data_cleaned.csv', encoding='utf-8', sep=';', index=False)